In [1]:
import matplotlib.pyplot as plt
import csv
import os

import villas.dataprocessing.readtools as rt
import villas.dataprocessing.timeseries as ts
import villas.dataprocessing.plottools as pt
import villas.dataprocessing.validationtools as vt


### Read DPsim files

In [2]:
# Set path to file
path = 'logs/WSCC-9bus_cosim_extension/'
fileName = 'WSCC-9bus_cosim_extension'
resultFile = path + fileName + '.csv'

# Returns list of timeseries-objects
ts_dpsim = rt.read_timeseries_dpsim(resultFile)

column number: 19
results length: 100
real column names: ['P_Load1', 'P_Load2', 'P_Load3', 'Q_Load1', 'Q_Load2', 'Q_Load3', 'Vnom_Load1', 'Vnom_Load2', 'Vnom_Load3']
complex column names: ['I_Load1', 'I_Load2', 'I_Load3', 'I_line1', 'I_line2', 'I_line3', 'v_node1', 'v_node2', 'v_node3', 'v_node4']


### Read DistAIX files

In [3]:
# Change file to distaix path
fileName = '3_4'
resultFile = path + fileName + '.csv'
resultFileFitted = path + fileName + '_temp.csv'

As DistAIX uses different headers, those have to be adapted to be used by villas.dataprocessing

In [4]:
with open(resultFile, 'r') as inFile, open(resultFileFitted, 'w') as outFile:
    read = csv.reader(inFile)
    write = csv.writer(outFile)
    
    new_header = []
    row = next(read)

    new_header.append('time')
    for element in row[1:]:
        for idx,char in enumerate(element):
            if char is '[':
                break
        if idx is len(element)-1:
            idx += 1
        new_header.append(element[:idx].strip())
        
    write.writerow(new_header)
    for row in read:
        write.writerow(row)

In [5]:
ts_distaix = rt.read_timeseries_csv(resultFileFitted)
os.remove(resultFileFitted)

column number: 4
results length: 100
real column names: ['Slosses', 'Srel']
complex column names: ['i1', 'i2']


### Compare the wished timeseries created by DPsim and DistAIX and calculate the error

In [6]:
ts1 = ts_dpsim['I_Load2']
ts2 = ts_distaix['i2']

rmse = ts1.rmse(ts1,ts2)
print(rmse)

diff = ts1.diff('diff', self, ts2)
print(diff)

plt.plot(diff.values.real)

(8.162707831641361-4.1466592502669855j)


NameError: name 'self' is not defined